# Embeddings with Cosine Similarity

In [1]:
import os, sys

import numpy as np
import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from metrics import Metrics
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction, TfidfFeatureExtraction, SentenceTransformerFeatureExtraction, BertFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read csv files and load as df

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/pr

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [5]:
pred_obser_df = pd.DataFrame()
pred_obser_df['Prediction'] = predictions_df['Base Sentence']
pred_obser_df['Observation'] = observations_df['Base Sentence']
pred_obser_df.head(7)

,Prediction,Observation
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise."
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","A young investor predicts on 2025-03-15, the S&P 500 index may rise."
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026."
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.","In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable."
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.","The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase."
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,JPMorgan observed that the net profit at Microsoft had risen in September 2023.


## Extract Embeddings with Spacy

In [6]:
spacy_fe_pred = SpacyFeatureExtraction(pred_obser_df, 'Prediction')
spacy_embeddings_pred = spacy_fe_pred.sentence_feature_extraction()

spacy_fe_obser = SpacyFeatureExtraction(pred_obser_df, 'Observation')
obser_embeddings_pred = spacy_fe_obser.sentence_feature_extraction()

spacy_cs_metrics = Metrics.get_cosine_similarity(spacy_embeddings_pred, obser_embeddings_pred)
pred_obser_df['Spacy'] = spacy_cs_metrics
pred_obser_df

100%|██████████| 694/694 [00:00<00:00, 10934.15it/s]


,Prediction,Observation,Spacy
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0.863931
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0.822511
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0.918182
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0.900307
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.","In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0.839434
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.","The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0.901941
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0.888222
7,"On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.","On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",0.951855
8,"Bank of America predicts on 2024-08-21, the operating income at Visa may rise.","Bank of America predicts on October 15, 2024, the operating income at Johnson & Johnson may rise.",0.923571
9,"According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2029 Q2.","According to Google, the gross profit at Facebook would fall in the second quarter of 2026.",0.886981


## Extract Embeddings with Sentence Transformer

In [7]:
st_fe_pred = SentenceTransformerFeatureExtraction(pred_obser_df, 'Prediction')
st_embeddings_pred = st_fe_pred.sentence_feature_extraction()

st_fe_obser = SentenceTransformerFeatureExtraction(pred_obser_df, 'Observation')
st_embeddings_obser = st_fe_obser.sentence_feature_extraction()

st_cs_metrics = Metrics.get_cosine_similarity(st_embeddings_pred, st_embeddings_obser)
pred_obser_df['SentenceTransformer'] = st_cs_metrics
pred_obser_df

  0%|          | 0/694 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 1/694 [00:00<01:11,  9.63it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 694/694 [00:00<00:00, 10701.80it/s]


,Prediction,Observation,Spacy,SentenceTransformer
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0.863931,0.398911
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0.822511,0.541872
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0.918182,0.533953
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0.900307,0.515507
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.","In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0.839434,0.549199
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.","The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0.901941,0.501962
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0.888222,0.700528
7,"On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.","On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",0.951855,0.647421
8,"Bank of America predicts on 2024-08-21, the operating income at Visa may rise.","Bank of America predicts on October 15, 2024, the operating income at Johnson & Johnson may rise.",0.923571,0.751232
9,"According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2029 Q2.","According to Google, the gross profit at Facebook would fall in the second quarter of 2026.",0.886981,0.581357


## Extract Embeddings with Bert 

In [9]:
bert_fe_pred = BertFeatureExtraction(pred_obser_df, 'Prediction')
bert_embeddings_pred = bert_fe_pred.sentence_feature_extraction()

bert_fe_obser = BertFeatureExtraction(pred_obser_df, 'Observation')
bert_embeddings_obser = bert_fe_obser.sentence_feature_extraction()

bert_cs_metrics = Metrics.get_cosine_similarity(bert_embeddings_pred, bert_embeddings_obser)
pred_obser_df['BERT'] = bert_cs_metrics
pred_obser_df

100%|██████████| 694/694 [00:00<00:00, 11083.24it/s]


,Prediction,Observation,Spacy,SentenceTransformer,BERT
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0.863931,0.398911,0.917585
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0.822511,0.541872,0.895553
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0.918182,0.533953,0.895016
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0.900307,0.515507,0.948904
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.","In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0.839434,0.549199,0.901743
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.","The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0.901941,0.501962,0.955519
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0.888222,0.700528,0.941328
7,"On August 25, 2024, to September 25, 2025, Citigroup speculates the net profit at Johnson & Johnson will likely increase.","On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",0.951855,0.647421,0.955662
8,"Bank of America predicts on 2024-08-21, the operating income at Visa may rise.","Bank of America predicts on October 15, 2024, the operating income at Johnson & Johnson may rise.",0.923571,0.751232,0.961770
9,"According to Goldman Sachs, the research and development expenses at Alphabet would fall in 2029 Q2.","According to Google, the gross profit at Facebook would fall in the second quarter of 2026.",0.886981,0.581357,0.926305
